In [ ]:
#build a  model that predicts if someone who 
#seeks a loan might be a defaulter or a non defaulter

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree


In [2]:
data=pd.read_csv('credit.csv')

In [3]:
data.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   checking_balance      1000 non-null   object
 1   months_loan_duration  1000 non-null   int64 
 2   credit_history        1000 non-null   object
 3   purpose               1000 non-null   object
 4   amount                1000 non-null   int64 
 5   savings_balance       1000 non-null   object
 6   employment_duration   1000 non-null   object
 7   percent_of_income     1000 non-null   int64 
 8   years_at_residence    1000 non-null   int64 
 9   age                   1000 non-null   int64 
 10  other_credit          1000 non-null   object
 11  housing               1000 non-null   object
 12  existing_loans_count  1000 non-null   int64 
 13  job                   1000 non-null   object
 14  dependents            1000 non-null   int64 
 15  phone                 1000 non-null   o

In [5]:
data.shape

(1000, 17)

In [6]:
data.isnull().values.any()

np.False_

In [7]:
#convert object type into categorical datatype
#target--- defaulter

In [8]:
data.drop('phone',axis=1,inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   checking_balance      1000 non-null   object
 1   months_loan_duration  1000 non-null   int64 
 2   credit_history        1000 non-null   object
 3   purpose               1000 non-null   object
 4   amount                1000 non-null   int64 
 5   savings_balance       1000 non-null   object
 6   employment_duration   1000 non-null   object
 7   percent_of_income     1000 non-null   int64 
 8   years_at_residence    1000 non-null   int64 
 9   age                   1000 non-null   int64 
 10  other_credit          1000 non-null   object
 11  housing               1000 non-null   object
 12  existing_loans_count  1000 non-null   int64 
 13  job                   1000 non-null   object
 14  dependents            1000 non-null   int64 
 15  default               1000 non-null   o

In [10]:
for i in data.columns:
    if(data[i].dtype=='object'):
        data[i]=pd.Categorical(data[i])

In [11]:
#checking unknown values in a column
print(data.checking_balance.value_counts())

checking_balance
unknown       394
< 0 DM        274
1 - 200 DM    269
> 200 DM       63
Name: count, dtype: int64


In [12]:
print(data.credit_history.value_counts())

credit_history
good         530
critical     293
poor          88
very good     49
perfect       40
Name: count, dtype: int64


In [13]:
#checking unknown values in a column
print(data.purpose.value_counts())

purpose
furniture/appliances    473
car                     337
business                 97
education                59
renovations              22
car0                     12
Name: count, dtype: int64


In [14]:
#checking unknown values in a column
print(data.savings_balance.value_counts())

savings_balance
< 100 DM         603
unknown          183
100 - 500 DM     103
500 - 1000 DM     63
> 1000 DM         48
Name: count, dtype: int64


In [15]:
#checking unknown values in a column
print(data.employment_duration.value_counts())

employment_duration
1 - 4 years    339
> 7 years      253
4 - 7 years    174
< 1 year       172
unemployed      62
Name: count, dtype: int64


In [16]:
#checking unknown values in a column
print(data.other_credit.value_counts())

other_credit
none     814
bank     139
store     47
Name: count, dtype: int64


In [17]:
#checking unknown values in a column
print(data.housing.value_counts())

housing
own      713
rent     179
other    108
Name: count, dtype: int64


In [18]:
#checking unknown values in a column
print(data.job.value_counts())

job
skilled       630
unskilled     200
management    148
unemployed     22
Name: count, dtype: int64


In [19]:
replace_data={'checking_balance':{"< 0 DM":1,"1 - 200 DM":2,
                                 "> 200 DM":3,"unknown":-1},
             'credit_history':{"critical":1,"poor":2,
                              "good":3,"very good":4,"perfect":5},
             #"Savings_balance":{"< 100 DM":1,"100 - 500 DM":2,
                              # "500 - 1000 DM":3,"> 1000 DM":4,
                              # "unknown":-1},
             "employment_duration":{'unemployed':1,'< 1 year':2,
                                   "1 - 4 years":3,
                                   "4 - 7 years":4,"> 7 years":5},
             "job":{"unemployed":1,"unskilled":2,"skilled":3,
                   "management":4},
             "default":{"no":0,"yes":1}}

In [20]:
data=data.replace(replace_data)

C:\Users\raksh\AppData\Local\Temp\ipykernel_2672\7625557.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data=data.replace(replace_data)
C:\Users\raksh\AppData\Local\Temp\ipykernel_2672\7625557.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data=data.replace(replace_data)
C:\Users\raksh\AppData\Local\Temp\ipykernel_2672\7625557.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future 

In [21]:
one_hot_cols=['purpose','housing','other_credit','savings_balance']

In [22]:
data=pd.get_dummies(data,columns=one_hot_cols)

In [23]:
x=data.drop('default',axis=1)
y=data['default']

In [24]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

In [25]:
model=DecisionTreeClassifier(random_state=1)

In [26]:
model.fit(x_train,y_train)

DecisionTreeClassifier(random_state=1)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   checking_balance               1000 non-null   category
 1   months_loan_duration           1000 non-null   int64   
 2   credit_history                 1000 non-null   category
 3   amount                         1000 non-null   int64   
 4   employment_duration            1000 non-null   category
 5   percent_of_income              1000 non-null   int64   
 6   years_at_residence             1000 non-null   int64   
 7   age                            1000 non-null   int64   
 8   existing_loans_count           1000 non-null   int64   
 9   job                            1000 non-null   category
 10  dependents                     1000 non-null   int64   
 11  default                        1000 non-null   category
 12  purpose_business               1000

In [28]:
#overfitting model
#Decision tree always tends to overfit
#overcome-----pruning
#Reducing overfitting technique------regularization
#Regularization technique used with decision tree----pruning

In [29]:
#pruning in decision trees is used to reduce the size of the
#tree and improve its generalization ability

In [30]:
#02/07/2024

#Gini Impurity : Measures the likelihood of incorrectly classifying a randomly chosen element
#from the data set if it was randomly labeled according to the data of labels in the node

model1=DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=1) 

In [31]:
model1.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=3, random_state=1)

In [32]:
print(model1.score(x_train,y_train))
print(model1.score(x_test,y_test))

0.7571428571428571
0.75


In [33]:
y_predict=model1.predict(x_test)

In [34]:
#confusion matrix.here we take 0 and  1 as default
print("Confusion matrix")
confusion_matrix_data=metrics.confusion_matrix(y_test,y_predict,labels=[1,0])

Confusion matrix


In [35]:
d=pd.DataFrame(confusion_matrix_data,index=[i for i in['Yes','No']],
               columns=[i for i in['Yes','No']])

In [36]:
print(d)

     Yes   No
Yes   34   52
No    23  191


In [37]:
tp=34
fn=52
tn=191
fp=23

In [38]:
accuracy=((tp+tn)/(tp+tn+fp+fn))
print("Accuracy of Model=",accuracy)

Accuracy of Model= 0.75


In [ ]:
##########################################################

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
#Instantiate the class
#by changing the estimator value we can change the accuracy of test data
model2=RandomForestClassifier(n_estimators=25,random_state=1,max_features=10)

In [41]:
#Fit the model
model2.fit(x_train,y_train)

RandomForestClassifier(max_features=10, n_estimators=25, random_state=1)

In [42]:
print("Accuracy on training set:",model2.score(x_train,y_train))
print("Accuarcy on test set:",model2.score(x_test,y_test))

Accuracy on training set: 1.0
Accuarcy on test set: 0.76


In [43]:
result=pd.DataFrame(model1.feature_importances_,columns=["Imp"],index=x_train.columns)

In [44]:
result

,Imp
checking_balance,0.485111
months_loan_duration,0.167255
credit_history,0.163614
amount,0.000000
employment_duration,0.000000
percent_of_income,0.000000
years_at_residence,0.000000
age,0.000000
existing_loans_count,0.000000
job,0.000000


In [ ]:
##########################################################

In [45]:
#July-04
from sklearn.ensemble import AdaBoostClassifier

In [46]:
ada_boost_model=AdaBoostClassifier(n_estimators=10,random_state=10)

In [47]:
ada_boost_model.fit(x_train,y_train)

C:\Users\raksh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(n_estimators=10, random_state=10)

In [48]:
print(ada_boost_model.score(x_train,y_train))
print(ada_boost_model.score(x_test,y_test))

0.7742857142857142
0.7433333333333333


In [49]:
ada_boost_predict=ada_boost_model.predict(x_test)

In [59]:
print("Confusion matrix")
confusion_matrix_ada_boost=metrics.confusion_matrix(y_test,ada_boost_predict,labels=[1,0])

Confusion matrix


In [60]:
d=pd.DataFrame(confusion_matrix_ada_boost,index=[i for i in['Yes','No']],
               columns=[i for i in['Yes','No']])

In [61]:
print(d)

     Yes   No
Yes   30   56
No    21  193


In [62]:
tp=30
tn=193
fp=21
fn=56

In [63]:
accuracy=((tp+tn)/(tp+tn+fp+fn))
print("Accuracy of Model=",accuracy)

Accuracy of Model= 0.7433333333333333


In [ ]:
#############################################################

In [64]:
from sklearn.ensemble import GradientBoostingClassifier

In [66]:
gradient_model=GradientBoostingClassifier(n_estimators=50,random_state=1)

In [67]:
gradient_model.fit(x_train,y_train)

GradientBoostingClassifier(n_estimators=50, random_state=1)

In [68]:
print(gradient_model.score(x_train,y_train))
print(gradient_model.score(x_test,y_test))

0.8671428571428571
0.7466666666666667


In [69]:
gradient_predict=gradient_model.predict(x_test)

In [70]:
confusion_matrix_gradient=metrics.confusion_matrix(y_test,gradient_predict,labels=[1,0])

In [71]:
d=pd.DataFrame(confusion_matrix_gradient,index=[i for i in['Yes','No']],
               columns=[i for i in['Yes','No']])

In [72]:
print(d)

     Yes   No
Yes   28   58
No    18  196


In [73]:
tp=28
tn=196
fp=18
fn=58

In [76]:
accuracy=(tp+tn)/(tp+fp+tn+fn)
print(accuracy)

0.7466666666666667


In [77]:
#Decision Tree-------75
#Random Forest-------76
#Ada Boost---------74
#Gradient Boost------74